In [1]:
# !pip uninstall tidyzoning -y
# !pip install git+https://github.com/HOUPU1993/tidyzoning.git

  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-mxe5umee
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit a421a514cf85703cdbbaf227597771dc4c809c96
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tidyzoning: filename=tidyzoning-0.1.0-py3-none-any.whl size=4404 sha256=598256b7f4bd4ee64fe94feabc0016f3554cc2c70991d63d44c39e9641db0ad4
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-fkdang4h\wheels\f4\29\51\e431eb95e0bf6589310375ce483daada9c916a6f65f3b65a8f
Successfully built tidyzoning


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-mxe5umee'


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json

from shapely.geometry import Point
from shapely.geometry import box

# input my libraries
from tidyzoning import find_district_idx
from tidyzoning import find_bldg_type
from tidyzoning import check_land_use

In [2]:
tidybuilding = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data/Tidybuildings/tidybuilding_2family.geojson")
tidyparcel = gpd.read_file(r"C:/Users/Admin/Desktop/zoning_data/Test_Data/Test_Parcels/testparcel_100per_correct.geojson")
tidyzoning = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data//Tidyzoning/tidyzoning_Haltom_City.geojson")

tidyzoning = tidyzoning.to_crs(tidyparcel.crs)
tidybuilding = tidybuilding.to_crs(tidyparcel.crs)

## Create Different Functions

### `find_district_idx` function

In [3]:
find_district_idx_results = find_district_idx(tidyparcel, tidyzoning)
find_district_idx_results

,prop_id,object_id,zoning_id
3,03027805,1000,1.0
8,03027783,1001,1.0
10,03027791,1002,1.0
18,00684619,1003,3.0
22,06687849,1004,3.0
...,...,...,...
75858,03027848,995,1.0
75863,03027856,996,1.0
75868,03027821,997,1.0
75871,04723295,998,1.0


### `find_bldg_type` function

In [4]:
find_bldg_type_results = find_bldg_type(tidybuilding)
find_bldg_type_results

'2_family'

### `get_zoning_req` function  
- `find_bldg_type` functiona is integrated internally.

In [5]:
tidybuilding

,units_3bed,total_floors,max_unit_size,min_unit_size,floor_area,building_height,geometry
0,2.0,3.0,1563.0,1563.0,3126.0,45.0,"POLYGON ((-11797996.293 383429.969, -11797986...."


In [6]:
tidyzoning

,dist_info,lot_constraints,structure_constraints,other_constraints,geometry
0,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829182.585 3866215.182, -10..."
1,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829727.287 3866115.173, -10..."
2,"{'dist_name': 'Manufactured Housing', 'dist_ab...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...",None
3,"{'dist_name': 'Duplex Residential', 'dist_abbr...","{'lot_size': [{'unit': 'acres', 'use_name': ['...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10829844.874 3866386.842, -10..."
4,"{'dist_name': 'Townhouse Residential', 'dist_a...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...",None
5,"{'dist_name': 'Multiple Family Residential', '...",[ ],"{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...",None
6,"{'dist_name': 'Multiple Family Residential', '...",[ ],"{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10827770.055 3873318.245, -10..."
7,"{'dist_name': 'Office', 'dist_abbr': 'O', 'dis...",[ ],[ ],[ ],"MULTIPOLYGON (((-10830277.539 3866854.440, -10..."
8,"{'dist_name': 'Commercial', 'dist_abbr': 'C-1'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828680.964 3866302.949, -10..."
9,"{'dist_name': 'Commercial', 'dist_abbr': 'C-2'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828928.601 3866165.418, -10..."


In [6]:
tidyparcel

,Prop_ID,GEO_ID,parcel_id,parcel_addr,landuse,landuse_spec,parcel_label,Found_Match,match_road_address,shape_index,50_threshold,num_edges,angle_difference,shared_side,parcel_bearing,road_bearing,angle,distance_to_road,side,geometry
0,03027805,NaN,1000,3924 EARL ST,R,A,regular inside parcel,True,Earl St,1.192563,False,4,0.790292,False,-0.001866,0.011927,0.790292,7.329586,front,"LINESTRING (-10830196.724 3866975.583, -108301..."
1,03027805,NaN,1000,3924 EARL ST,R,A,regular inside parcel,True,Earl St,1.192563,False,4,0.790292,True,1.560608,0.011927,88.732853,28.968721,Interior side,"LINESTRING (-10830197.168 3866931.986, -108301..."
2,03027805,NaN,1000,3924 EARL ST,R,A,regular inside parcel,True,Earl St,1.192563,False,4,0.790292,True,-3.138026,0.011927,0.479009,50.845007,rear,"LINESTRING (-10830175.077 3866932.065, -108301..."
3,03027805,NaN,1000,3924 EARL ST,R,A,regular inside parcel,True,Earl St,1.192563,False,4,0.790292,True,-1.589407,0.011927,88.250309,29.215929,centroid,POINT (-10830185.813 3866953.859)
4,03027805,NaN,1000,3924 EARL ST,R,A,regular inside parcel,True,Earl St,1.192563,False,4,0.790292,True,-1.589407,0.011927,88.250309,29.215929,Interior side,"LINESTRING (-10830174.268 3866975.541, -108301..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75875,00381004,NaN,999,1712 N BEACH ST,NaN,F1,regular inside parcel,True,N Beach St,1.361853,True,4,0.044117,True,3.141498,1.573211,89.856205,79.090904,Interior side,"LINESTRING (-10830195.409 3866894.266, -108302..."
75876,00381004,NaN,999,1712 N BEACH ST,NaN,F1,regular inside parcel,True,N Beach St,1.361853,True,4,0.044117,True,-1.558978,1.573211,0.538781,145.821717,centroid,POINT (-10830262.414 3866913.126)
75877,00381004,NaN,999,1712 N BEACH ST,NaN,F1,regular inside parcel,True,N Beach St,1.361853,True,4,0.044117,True,-1.558978,1.573211,0.538781,145.821717,rear,"LINESTRING (-10830195.855 3866932.011, -108301..."
75878,00381004,NaN,999,1712 N BEACH ST,NaN,F1,regular inside parcel,True,N Beach St,1.361853,True,4,0.044117,True,-0.000172,1.573211,89.851809,78.928026,Interior side,"LINESTRING (-10830329.288 3866932.034, -108303..."


In [23]:
tidyzoning['structure_constraints'].loc[0]

{'setback_front': [{'unit': 'feet',
   'use_name': '1_family',
   'min_val': {'expression': '25'}}],
 'setback_side_int': [{'unit': 'feet',
   'use_name': '1_family',
   'min_val': {'expression': '7.5'}}],
 'setback_side_ext': [{'unit': 'feet',
   'use_name': '1_family',
   'min_val': {'expression': '25'}}],
 'setback_rear': [{'unit': 'feet',
   'use_name': '1_family',
   'min_val': {'expression': '20'}}],
 'lot_cov_bldg': [{'unit': 'percent',
   'use_name': '1_family',
   'max_val': {'expression': '50'}}],
 'height': [{'unit': 'feet',
   'use_name': '1_family',
   'max_val': {'expression': '35'}}],
 'unit_size': [{'unit': 'square feet',
   'use_name': 'ADU',
   'min_val': {'expression': '200'}},
  {'unit': 'square feet',
   'use_name': '1_family',
   'min_val': {'expression': '2000'}}]}

In [24]:
tidyzoning['other_constraints'].loc[0]

{'parking': [{'unit': 'stalls',
   'use_name': '1_family',
   'min_val': {'expression': '2'}}]}

In [26]:
tidyzoning["lot_constraints"].loc[0]

{'lot_size': [{'unit': 'acres',
   'use_name': '1_family',
   'min_val': {'expression': '0.17'}}]}

In [7]:
# 测试第一段
# 提取 district constraints 数据
columns_to_extract = ['structure_constraints', 'other_constraints', 'lot_constraints']
extracted_data = []

for col in columns_to_extract:
    for index, row in tidyzoning.iterrows():
        constraints = row[col]
        if isinstance(constraints, dict):  # 确保是字典
            for constraint_type, entries in constraints.items():
                if isinstance(entries, list):  # 遍历键值对应的列表
                    for entry in entries:
                        flattened_entry = {
                            "original_index": index,
                            "source_column": col,
                            "constraint_type": constraint_type,
                        }
                        # 将 entry 中的所有键值对展开
                        for key, value in entry.items():
                            if isinstance(value, dict):  # 对嵌套字典再展开
                                for sub_key, sub_value in value.items():
                                    flattened_entry[f"{key}_{sub_key}"] = sub_value
                            else:
                                flattened_entry[key] = value

                        # 添加结果到列表
                        extracted_data.append(flattened_entry)

# 转换为 DataFrame
district_constraints = pd.DataFrame(extracted_data)
district_constraints

,original_index,source_column,constraint_type,unit,use_name,min_val_expression,max_val_expression
0,0,structure_constraints,setback_front,feet,1_family,25,NaN
1,0,structure_constraints,setback_side_int,feet,1_family,7.5,NaN
2,0,structure_constraints,setback_side_ext,feet,1_family,25,NaN
3,0,structure_constraints,setback_rear,feet,1_family,20,NaN
4,0,structure_constraints,lot_cov_bldg,percent,1_family,NaN,50
...,...,...,...,...,...,...,...
59,0,lot_constraints,lot_size,acres,1_family,0.17,NaN
60,1,lot_constraints,lot_size,acres,1_family,0.16,NaN
61,2,lot_constraints,lot_size,acres,1_family,0.18,NaN
62,3,lot_constraints,lot_size,acres,"[1_family, 2_family]",0.19,NaN


In [34]:
from shapely.ops import unary_union
from shapely.geometry import Polygon
from shapely.ops import polygonize

# 检查 tidyparcel 是否存在并运行主逻辑
if tidyparcel is None:
    lot_width = None
    lot_depth = None
    lot_area = None
else:
    # 按地块处理
    results = []

    # 遍历每个地块
    for prop_id, group in tidyparcel.groupby('Prop_ID'):
        # 筛选不同边的信息
        front_of_parcel = group[group['side'] == "front"]
        side_of_parcel = group[group['side'] == "Interior side"]
        parcel_without_centroid = group[(group['side'].notna()) & (group['side'] != "centroid")]

        # 计算宽度和深度
        lot_width = front_of_parcel.geometry.length.sum() * 3.28084  # 转换为英尺
        lot_depth = side_of_parcel.geometry.length.sum() * 3.28084  # 转换为英尺

        # 合并地块边界，并计算面积
        polygons = polygonize(unary_union(parcel_without_centroid.geometry))
        lot_polygon = unary_union(polygons) 
        lot_area = lot_polygon.area * 10.7639  # 转换为平方英尺
        
        # 保存每个地块的结果
        results.append({
            "Prop_ID": prop_id,
            "lot_width": lot_width,
            "lot_depth": lot_depth,
            "lot_area": lot_area,
        })

# 转换为 DataFrame 以便后续使用
parcel_results = pd.DataFrame(results)
parcel_results

,Prop_ID,lot_width,lot_depth,lot_area
0,00011576,92.373401,818.171822,38041.440375
1,00011606,85.572510,408.175712,17585.776009
2,00011614,96.258861,409.680261,19718.034628
3,00011622,90.327104,102.579833,9455.556442
4,00011630,102.568901,187.189395,9603.590021
...,...,...,...,...
15172,43063724,644.255301,1388.306390,894151.041876
15173,43083679,67.972160,299.422590,10187.738495
15174,43083687,65.618534,299.529779,9824.700210
15175,43097009,413.525737,770.005362,159319.950888


In [49]:
tidybuilding

,units_3bed,total_floors,max_unit_size,min_unit_size,floor_area,building_height,geometry
0,2.0,3.0,1563.0,1563.0,3126.0,45.0,"POLYGON ((-11797996.293 383429.969, -11797986...."


In [51]:
fl_area = tidybuilding.get('floor_area', 0)
fl_area

0    3126.0
Name: floor_area, dtype: float64

In [111]:
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union
from shapely.geometry import Polygon
from shapely.ops import polygonize

def get_zoning_req(tidybuilding, tidyzoning, tidyparcel=None):
    """
    处理建筑、区划和地块信息，生成结构化的 Zoning 需求数据。
    
    :param tidybuilding: 建筑数据 (DataFrame)
    :param tidyzoning: 区划数据 (DataFrame)
    :param tidyparcel: 地块数据 (GeoDataFrame)，可选
    :return: 一个字典，包含所有计算的结果
    """
    # 提取 district constraints 数据
    columns_to_extract = ['structure_constraints', 'other_constraints', 'lot_constraints']
    extracted_data = []

    for col in columns_to_extract:
        for index, row in tidyzoning.iterrows():
            constraints = row[col]
            if isinstance(constraints, dict):  # 确保是字典
                for constraint_type, entries in constraints.items():
                    if isinstance(entries, list):  # 遍历键值对应的列表
                        for entry in entries:
                            flattened_entry = {
                                "original_index": index,
                                "source_column": col,
                                "constraint_type": constraint_type,
                            }
                            # 将 entry 中的所有键值对展开
                            for key, value in entry.items():
                                if isinstance(value, dict):  # 对嵌套字典再展开
                                    for sub_key, sub_value in value.items():
                                        flattened_entry[f"{key}_{sub_key}"] = sub_value
                                else:
                                    flattened_entry[key] = value
                            # 添加结果到列表
                            extracted_data.append(flattened_entry)
    # 转换为 DataFrame
    district_constraints = pd.DataFrame(extracted_data)


    # 检查 tidyparcel 是否存在并运行主逻辑
    if tidyparcel is None:
        lot_width = None
        lot_depth = None
        lot_area = None
    else:
        # 按地块处理
        results = []

        # 遍历每个地块
        for prop_id, group in tidyparcel.groupby('Prop_ID'):
            # 筛选不同边的信息
            front_of_parcel = group[group['side'] == "front"]
            side_of_parcel = group[group['side'] == "Interior side"]
            parcel_without_centroid = group[(group['side'].notna()) & (group['side'] != "centroid")]
            # 计算宽度和深度
            lot_width = front_of_parcel.geometry.length.sum() * 3.28084  # 转换为英尺
            lot_depth = side_of_parcel.geometry.length.sum() * 3.28084  # 转换为英尺
            # 合并地块边界，并计算面积
            polygons = polygonize(unary_union(parcel_without_centroid.geometry))
            lot_polygon = unary_union(polygons) 
            lot_area = lot_polygon.area * 10.7639  # 转换为平方英尺
            # 保存每个地块的结果
            results.append({
                "Prop_ID": prop_id,
                "lot_width": lot_width,
                "lot_depth": lot_depth,
                "lot_area": lot_area
            })
    # 转换为 DataFrame 以便后续使用
    parcel_results = pd.DataFrame(results)


    # 处理建筑数据
    bed_list = {
        'units_0bed': 0,
        'units_1bed': 1,
        'units_2bed': 2,
        'units_3bed': 3,
        'units_4bed': 4
    }

    bedrooms = max([bed_list.get(col, 0) for col in tidybuilding.columns if col in bed_list.keys()])

    units_0bed = tidybuilding.get('units_0bed', 0)
    units_1bed = tidybuilding.get('units_1bed', 0)
    units_2bed = tidybuilding.get('units_2bed', 0)
    units_3bed = tidybuilding.get('units_3bed', 0)
    units_4bed = tidybuilding.get('units_4bed', 0)

    total_units = units_0bed + units_1bed + units_2bed + units_3bed + units_4bed

    fl_area = tidybuilding.get('floor_area', [None])[0]
    parking_open = tidybuilding.get('parking_open', [None])[0]
    parking_enclosed = tidybuilding.get('parking_enclosed', [None])[0]
    parking = tidybuilding.get('parking', [None])[0]
    height = tidybuilding.get('building_height', [None])[0]
    floors = tidybuilding.get('total_floors', [None])[0]
    min_unit_size = tidybuilding.get('min_unit_size', [None])[0]
    max_unit_size = tidybuilding.get('max_unit_size', [None])[0]

    far = fl_area / lot_area if lot_area else None

    # 汇总结果
    results = {
        "district_constraints": district_constraints,
        "lot_width": lot_width,
        "lot_depth": lot_depth,
        "lot_area": lot_area,
        "bedrooms": bedrooms,
        "total_units": total_units,
        "fl_area": fl_area,
        "parking_open": parking_open,
        "parking_enclosed": parking_enclosed,
        "parking": parking,
        "height": height,
        "floors": floors,
        "min_unit_size": min_unit_size,
        "max_unit_size": max_unit_size,
        "far": far
    }

    return results

In [112]:
result = get_zoning_req(tidybuilding, tidyzoning, tidyparcel)
result

{'district_constraints':     original_index          source_column   constraint_type     unit  \
 0                0  structure_constraints     setback_front     feet   
 1                0  structure_constraints  setback_side_int     feet   
 2                0  structure_constraints  setback_side_ext     feet   
 3                0  structure_constraints      setback_rear     feet   
 4                0  structure_constraints      lot_cov_bldg  percent   
 ..             ...                    ...               ...      ...   
 59               0        lot_constraints          lot_size    acres   
 60               1        lot_constraints          lot_size    acres   
 61               2        lot_constraints          lot_size    acres   
 62               3        lot_constraints          lot_size    acres   
 63               4        lot_constraints          lot_size    acres   
 
                 use_name min_val_expression max_val_expression  
 0               1_family       

### `Check` function

#### 01. `check_land_use` function
- `find_bldg_type` functiona is integrated internally.

In [5]:
result_df = check_land_use(tidybuilding, tidyzoning)
result_df

,zoning_id,allowed
0,0,False
1,1,False
2,2,False
3,3,True
4,4,False
5,5,False
6,6,False
7,7,False
8,8,False
9,9,False


#### 02. `check_floors` function
- `get_zoning_req` functiona is integrated internally.